In [1]:
# Explore the columns of data and find outliers
import pandas as pd
import numpy as np

In [3]:
charts_df = pd.read_csv('../../data/charts_processed.csv')
audio_df = pd.read_csv('../../data/audio_features_processed.csv')

In [4]:
charts_df.head()

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0


In [6]:
# Make the index the id for a faster accessing
audio_df.set_index('id', inplace=True)

audio_df.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
id,,,,,,,,,,,,,
6mICuAdrwEjh6Y6lroV2Kg,0.18700,0.852,195840,0.773,0.000030,8,0.1590,-2.921,0,0.0776,102.034,4,0.907
7DM4BPaS7uofFul3ywMe46,0.00431,0.663,259196,0.920,0.000017,11,0.1010,-4.070,0,0.2260,99.935,4,0.533
3AEZUABDXNtecAOSC1qTfo,0.40000,0.761,222560,0.838,0.000000,4,0.1760,-3.073,0,0.0502,93.974,4,0.710
6rQSrBHf7HlZjtcMZ4S4bO,0.55100,0.508,205600,0.687,0.000003,0,0.1260,-4.361,1,0.3260,180.044,4,0.555
58IL315gMSTD37DOZPJ2hf,0.07600,0.899,234320,0.626,0.000000,6,0.0631,-4.228,0,0.2920,88.007,4,0.873


# Datetime process

In [ ]:
from datetime import datetime

# Convert the date column to datetime.date
charts_df['date'] = charts_df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())

def get_charts_by_date(date):
    """
    If the date is a tuple, return the charts between the two dates
    """
    if isinstance(date,tuple):
        return charts_df[(charts_df['date'] >= date[0]) & (charts_df['date'] <= date[1])]
    else:
        return charts_df[charts_df['date'] == date]

